In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [129]:
df = pd.read_csv("./CSV_Files/glucose_data_resampled.csv")
# Drop all the columns which have unnamed in them
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,Glucose_time,reading
0,2023-03-05 00:05:00,115.0
1,2023-03-05 00:10:00,115.0
2,2023-03-05 00:15:00,114.0
3,2023-03-05 00:20:00,116.0
4,2023-03-05 00:25:00,116.5


In [130]:
# Set the Glucose_time to datetime format and set it as the index
df['Glucose_time'] = pd.to_datetime(df['Glucose_time'])
df.set_index('Glucose_time', inplace=True)
df.head()

,reading
Glucose_time,
2023-03-05 00:05:00,115.0
2023-03-05 00:10:00,115.0
2023-03-05 00:15:00,114.0
2023-03-05 00:20:00,116.0
2023-03-05 00:25:00,116.5


In [131]:
checked_df = df.copy()
scaler = MinMaxScaler(feature_range=(0, 1))
df['reading'] = scaler.fit_transform(df[['reading']])
df.head()

,reading
Glucose_time,
2023-03-05 00:05:00,0.840909
2023-03-05 00:10:00,0.840909
2023-03-05 00:15:00,0.818182
2023-03-05 00:20:00,0.863636
2023-03-05 00:25:00,0.875000


In [132]:
def prepare_data(time_series_data, n_features):
    X, y = [], []
    for i in range(len(time_series_data)):
        end_ix = i + n_features
        if end_ix > len(time_series_data)-1:
            break
        seq_x, seq_y = time_series_data[i:end_ix], time_series_data[end_ix]

        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [133]:
nfs = int(input("Enter the continuous inputs you want to give: "))

n_feat_list = []

for i in range(4, nfs+1):
    if i % 2 == 0:
        n_feat_list.append(i)

if nfs not in n_feat_list:
    n_feat_list.append(nfs)

n_feat_list

[4, 6, 8, 10]

In [134]:
patience_lst = [10, 15, 20, 25]

In [135]:
time_series_data = df['reading'].values

In [136]:
best_parameter = {}
best_rmse = float('inf')
best_predictions = None
best_model = None

In [137]:
for n_features in n_feat_list:
    for pat in patience_lst:
        X, y = prepare_data(time_series_data, n_features)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        test_size = 18
        val_size = 24
        train_size = X.shape[0] + (n_features - 5) - test_size - val_size

        X_train, y_train = X[:train_size], y[:train_size]
        X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
        X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

            # Print the shapes of the train, validation and test sets
        print("Train Shape: ", X_train.shape, y_train.shape)
        print("Validation Shape: ", X_val.shape, y_val.shape)
        print("Test Shape: ", X_test.shape, y_test.shape)

            
            # Building the LSTM Model
        model = Sequential()
        model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_features, 1)))
        model.add(LSTM(50, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

            # Early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

            # Fitting the model
        model.fit(X_train, y_train, epochs = 300, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

            # Choosing the best model based on the validation loss
        predictions = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, predictions))
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_parameter['n_features'] = n_features
            best_parameter['patience'] = pat
            best_predictions = predictions
            best_model = model

print("*" * 50)
print("Best Parameters: ", best_parameter)
print("Best RMSE: ", best_rmse)
print("Prediction :- ", best_predictions)


Train Shape:  (240, 4, 1) (240,)
Validation Shape:  (24, 4, 1) (24,)
Test Shape:  (19, 4, 1) (19,)


c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 20: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 835ms/step
Train Shape:  (240, 4, 1) (240,)
Validation Shape:  (24, 4, 1) (24,)
Test Shape:  (19, 4, 1) (19,)


c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 27: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Train Shape:  (240, 4, 1) (240,)
Validation Shape:  (24, 4, 1) (24,)
Test Shape:  (19, 4, 1) (19,)


...

Epoch 21: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Train Shape:  (240, 10, 1) (240,)
Validation Shape:  (24, 10, 1) (24,)
Test Shape:  (13, 10, 1) (13,)


c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 30: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 989ms/step
Train Shape:  (240, 10, 1) (240,)
Validation Shape:  (24, 10, 1) (24,)
Test Shape:  (13, 10, 1) (13,)


c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 31: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 853ms/step
**************************************************
Best Parameters:  {'n_features': 4, 'patience': 20}
Best RMSE:  0.04761749836465496
Prediction :-  [[0.337689  ]
 [0.34482515]
 [0.37528187]
 [0.39724836]
 [0.3871983 ]
 [0.4139311 ]
 [0.4344053 ]
 [0.3970126 ]
 [0.38764927]
 [0.37651494]
 [0.46868095]
 [0.46416026]
 [0.4745898 ]
 [0.52696174]
 [0.48586258]
 [0.44578168]
 [0.40453264]
 [0.35744718]
 [0.5219861 ]
 [0.6287734 ]
 [0.6472267 ]
 [0.6504896 ]
 [0.6572327 ]
 [0.51023555]]


In [138]:
validation_predictions_in_original_scale = scaler.inverse_transform(best_predictions)
xlst = validation_predictions_in_original_scale.flatten()
xlst

array([ 92.858315,  93.17231 ,  94.5124  ,  95.47892 ,  95.03672 ,
        96.21297 ,  97.11383 ,  95.46856 ,  95.056564,  94.56666 ,
        98.621956,  98.42305 ,  98.88195 , 101.18632 ,  99.377945,
        97.614395,  95.79944 ,  93.72768 , 100.96739 , 105.66603 ,
       106.47798 , 106.62154 , 106.91824 , 100.45036 ], dtype=float32)

In [139]:
actual_values = df['reading'].values[train_size:train_size+val_size]
actual_values = actual_values.reshape(-1, 1)
actual_values_in_original_scale = scaler.inverse_transform(actual_values)
actual_values_in_original_scale.flatten()

array([ 96.        ,  94.        ,  93.5       ,  93.        ,
        93.        ,  94.        ,  95.        ,  95.        ,
        96.        ,  97.        ,  96.        ,  95.5       ,
        95.        ,  98.        ,  98.5       ,  99.        ,
       101.        ,  99.66666667,  98.33333333,  97.        ,
        95.        , 100.        , 105.        , 106.        ])

In [140]:
# Now I want it to be printed in the form of a dataframe with the predicted values with a shift of 1 and the actual values

final = pd.DataFrame()
# Append the time of time series from values 240 to 264
final['time'] = df.index[train_size:train_size+val_size]


# Append the precited values with a shift of 1 and the predicted value at 240 being the mean of actual values at 237, 238 and 239. Append the mean first and then the predicted values
final['Shifted_prediction'] = [np.mean(actual_values_in_original_scale[237:240])] + validation_predictions_in_original_scale.flatten().tolist()[:-1]
final['unshifted_prediction'] = xlst.flatten().tolist()

# Append the actual values
final['actual'] = actual_values_in_original_scale.flatten()

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [141]:
final

,time,Shifted_prediction,unshifted_prediction,actual
0,2023-03-05 20:05:00,NaN,92.858315,96.000000
1,2023-03-05 20:10:00,92.858315,93.172310,94.000000
2,2023-03-05 20:15:00,93.172310,94.512398,93.500000
3,2023-03-05 20:20:00,94.512398,95.478920,93.000000
4,2023-03-05 20:25:00,95.478920,95.036720,93.000000
5,2023-03-05 20:30:00,95.036720,96.212967,94.000000
6,2023-03-05 20:35:00,96.212967,97.113831,95.000000
7,2023-03-05 20:40:00,97.113831,95.468559,95.000000
8,2023-03-05 20:45:00,95.468559,95.056564,96.000000
9,2023-03-05 20:50:00,95.056564,94.566658,97.000000


### This Cell Is For N Values = 1

In [79]:
# Now we have a value of Glucose Reading which is a single value. And we need to predict the next value of Glucose Reading. So, we will use the LSTM model to predict next 10 values of Glucose Reading
Reading_Given = [116.0]
Reading_Given_scaled = scaler.transform(np.array([Reading_Given]))

n_features = best_parameter['n_features']

# Now we will predict the next 10 values of Glucose Reading
X_new = Reading_Given_scaled
X_new = X_new.reshape((1, n_features, 1))
predictions = []
for i in range(10):
    pred = best_model.predict(X_new)
    predictions.append(pred)
    X_new = pred
    X_new = X_new.reshape((1, n_features, 1))

predictions = np.array(predictions)
predictions = predictions.reshape(-1, 1)
predictions_in_original_scale = scaler.inverse_transform(predictions)
predictions_in_original_scale.flatten()

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


array([116.27296 , 116.624115, 117.079094, 117.673996, 118.4611  ,
       119.518776, 120.96956 , 123.01547 , 126.01288 , 130.64818 ],
      dtype=float32)

### This Cell Is For N_Values > 1

In [42]:
# Given readings
Reading_Given = [116.0, 115.0, 118.0, 114.0, 115.75, 116.0]

# Scale the given readings
Reading_Given_scaled = scaler.transform(np.array(Reading_Given).reshape(-1, 1))

n_features = best_parameter['n_features']
# n_features = 2

# Now we will predict the next 10 values of Glucose Reading
X_new = Reading_Given_scaled
X_new = X_new.reshape((1, n_features, 1))
predictions = []

for i in range(10):
    pred = best_model.predict(X_new)
    predictions.append(pred)
    X_new = np.append(X_new, pred)
    X_new = X_new[-n_features:]
    X_new = X_new.reshape((1, n_features, 1))

predictions = np.array(predictions)
predictions = predictions.reshape(-1, 1)
predictions_in_original_scale = scaler.inverse_transform(predictions)
predictions_in_original_scale.flatten()


c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: cannot reshape array of size 5 into shape (1,2,1)

In [142]:
Reading_Given = [96, 94, 93.5, 93, 94, 95, 95, 96, 97, 96]
Reading_Given_scaled = scaler.transform(np.array(Reading_Given).reshape(-1, 1))

n_features = best_parameter['n_features']

X_new, _ = prepare_data(Reading_Given_scaled, n_features)

# Handle the case where not enough data is available to create the required sequence
if X_new.size == 0:
    # Pad the sequence with the initial readings (scaled)
    X_new = np.zeros((1, n_features, 1))
    X_new[0, :len(Reading_Given_scaled), 0] = Reading_Given_scaled.flatten()
else:
    # Reshape the data to the format expected by the LSTM model
    X_new = X_new.reshape((X_new.shape[0], n_features, 1))

# Ensure X_new is the correct shape for prediction
if X_new.shape[0] != 1:
    X_new = X_new[-1].reshape((1, n_features, 1))

predictions = []
for i in range(10):
    pred = best_model.predict(X_new)
    predictions.append(pred[0, 0])  # Append the first element of the prediction
    # Update X_new by removing the first value and adding the new prediction
    X_new = np.append(X_new[:, 1:, :], pred.reshape(1, 1, 1), axis=1)

# Inverse transform the predictions to get them back to the original scale
predictions_in_original_scale = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
predictions_in_original_scale = predictions_in_original_scale.flatten()

predictions_in_original_scale

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


array([97.11383 , 96.86169 , 96.42415 , 95.86365 , 95.30283 , 94.8166  ,
       94.45424 , 94.241455, 94.144104, 94.132034], dtype=float32)

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: cannot reshape array of size 8 into shape (1,4,1)